In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat, savemat
import pandas as pd
import copy

In [ ]:
M_all0 = np.loadtxt('M_all_asymmetric.csv')

In [ ]:
pop_usa = loadmat('../data/countypop_usafact.mat') ### assume the pop is corresponding to the fips 

In [ ]:
len(pop_usa['countypop'])

In [ ]:
pop = pop_usa['countypop'].ravel()

In [ ]:
fips = loadmat('../data/countyfips_num.mat')
fips_num = fips['countyfips_num'].ravel() 
fips_id = [int(i) for i in range(len(fips_num))]
# fips_dict = dict(zip(fips_num,fips_id))
df_fips = pd.DataFrame({'fips_num': fips_num, 'fips_id': fips_id, 'pop': pop_usa['countypop'].ravel()})

In [ ]:
no_fips = len(M_all0)
M_all = copy.deepcopy(M_all0)
C = np.zeros((no_fips,no_fips))
Cave = np.zeros((no_fips,no_fips))
countypop = np.zeros(no_fips)

In [ ]:
threshold = 100
for i in range(no_fips):
    M_i = M_all[:,i] 
    M_all[M_i < threshold,i] = 0 ## travel out <100 then set all the travelling to zero

In [145]:
for i in range(no_fips):
    pop_i = pop[i]
    travel_in = np.sum(M_all[i])
    within = M_all[i,i] ## people are travel within the fips
    travel_out = np.sum(M_all[:,i])
    
    if  pop_i - travel_out + within > within: ##
        C[i,i] = pop_i - travel_out + within ## why i need to add within??
        Cave[i,i] = pop_i - travel_out + within
    else:
        C[i,i] = within
        Cave[i,i] = within
        pop[i] = np.sum(M_all[:,i])
    
    for j in range(no_fips):
        if (M_all[j,i]>= threshold) and (M_all[i,j] >= threshold) and (j!=i):
            C[j,i]=M_all[j,i]
            Cave[j,i]=np.round((M_all[j,i]+M_all[i,j])/2+0.1) ## in python 0.5 round to 0/1, in matlab 0.5 round to 1

In [ ]:
## check if they sum up all the people travel out + stay_in will be equal to the populations

In [127]:
for i in range(no_fips):
    if np.sum(C[:,i]) != pop[i]:
        C[i,i] = pop[i] - np.sum(C[:,i]) + C[i,i]
    if np.sum(Cave[:,i]) != pop[i]:
        Cave[i,i] = pop[i] - np.sum(Cave[:,i]) + Cave[i,i]

In [ ]:
def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)

In [ ]:
print (check_symmetric(Cave), check_symmetric(C))

In [ ]:
print (no_fips**2 - np.sum(C == C.T), 'entries are not symmetric') ### 14450 entries are not symmetric
print (np.sum(C != 0), 'entries in the C') ### there are 17614 entries 

In [ ]:
C_matlab = loadmat('../data/commutedata_100.mat')

In [ ]:
part = C_matlab['part'].ravel()

In [ ]:
C_C = C_matlab['C'].ravel()

In [123]:
## check wether is same as the matlab results
for i in range(no_fips):
    start = part[i] - 1
    end = part[i+1] - 1
    Pei = C_C[start:end]
    dia = Pei[0]
    diamy = C[i,i]
    
    xx = C[C[:,i]!=0, i]
    my = np.delete(xx, np.where(xx==C[i,i]))
    
    if np.sum(Pei[1:] != my) >0:
        print (i)

2824
2829
2831
2835
2852
2896
2933
2945


In [124]:
C_Cave = C_matlab['Cave'].ravel()

In [148]:
for i in range(no_fips):
    start = part[i] - 1
    end = part[i+1] - 1
    Pei = C_Cave[start:end]
    dia = Pei[0]
    diamy = Cave[i,i]
    
    xx = Cave[Cave[:,i]!=0, i]
    my = np.delete(xx, np.where(xx==Cave[i,i]))
    
    if np.sum(Pei[1:] != my) >0:
        print (i, my)
        print(Pei[1:])
        print('*^^*')

2824 [ 641.  810.  464. 3545.]
[ 587  810  464 3545]
*^^*
2829 [ 641.  443. 1912.  377. 1260. 5725. 2869.]
[ 587  413 1763  306 1147 5468 2769]
*^^*
2831 [ 308.  443. 2029.  161. 3080.  753.]
[ 308  413 2029  161 3080  753]
*^^*
2835 [ 810.  472. 1912.  213. 1243. 6910.]
[ 810  472 1763  213 1243 6910]
*^^*
2852 [ 377.  954. 1014. 2355.  576.]
[ 306  954 1014 2355  576]
*^^*
2896 [ 1260.  2029.  1014.   923. 14004.  4292.]
[ 1147  2029  1014   923 14004  4292]
*^^*
2933 [3545. 1015. 5725. 6910.  636.]
[3545 1015 5468 6910  636]
*^^*
2945 [ 2869.  3080.  2355.  1072. 14004.   187.  4067.]
[ 2769  3080  2355  1072 14004   187  4067]
*^^*


In [153]:
np.savetxt('C.csv',C)
np.savetxt('Cave.csv',Cave)
np.savetxt('pop_new.csv', pop)